# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()


conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [7]:
conversation.predict(input="Hi, my name is Saban")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.
Human: Hi, my name is Saban
AI:

> Finished chain.


"Hello Saban! It's nice to meet you. How can I assist you today?"

In [8]:
conversation.predict(input="Who best footballer?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.
Human: Hi, my name is Saban
AI: Hello Saban! It's nice to meet you. How can I assist you today?
Human: Who best footballer?
AI:

> Finished chain.


"There are many great footballers, so it's subjective to determine who the best is. Some popular choices include Lionel Messi, Cristiano Ronaldo, and Neymar."

In [9]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.
Human: Hi, my name is Saban
AI: Hello Saban! It's nice to meet you. How can I assist you today?
Human: Who best footballer?
AI: There are many great footballers, so it's subjective to determine who the best is. Some popular choices include Lionel Messi, Cristiano Ronaldo, and Neymar.
Human: What is my name?
AI:

> Finished chain.


'Your name is Saban.'

In [10]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.
Human: Hi, my name is Saban
AI: Hello Saban! It's nice to meet you. How can I assist you today?
Human: Who best footballer?
AI: There are many great footballers, so it's subjective to determine who the best is. Some popular choices include Lionel Messi, Cristiano Ronaldo, and Neymar.
Human: What is my name?
AI: Your name is Saban.


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew.\nHuman: Hi, my name is Saban\nAI: Hello Saban! It's nice to meet you. How can I assist you today?\nHuman: Who best footballer?\nAI: There are many great footballers, so it's subjective to determine who the best is. Some popular choices include Lionel Messi, Cristiano Ronaldo, and Neymar.\nHuman: What is my name?\nAI: Your name is Saban."}

In [15]:
memory = ConversationBufferMemory()

memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})



{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [19]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)         

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [20]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)
#NOTE k=1 hafızayı sınırlandırdı
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

## ConversationTokenBufferMemory

In [25]:
#!pip install tiktoken
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

#NOTE sondan başlıyor kısıtlamaya
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [28]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=5)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': 'System: The human greets the AI and the AI asks what\'s up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about their schedule for the day. The AI informs them that they have a meeting at 8am with their product team, where they need to have their powerpoint presentation prepared. From 9am to 12pm, they have time to work on their LangChain project, which will go quickly due to LangChain\'s power. At noon, they have lunch at an Italian restaurant with a customer who is driving from over an hour away to learn about the latest in AI. The AI advises them to bring their laptop to show the latest LLM demo.'}

In [29]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="What would be a good demo to show?")
memory.load_memory_variables({})




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and the AI asks what's up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about their schedule for the day. The AI informs them that they have a meeting at 8am with their product team, where they need to have their powerpoint presentation prepared. From 9am to 12pm, they have time to work on their LangChain project, which will go quickly due to LangChain's power. At noon, they have lunch at an Italian restaurant with a customer who is driving from over an hour away to learn about the latest in AI. The AI advises them to bring their laptop to show the latest LLM demo.
Human: What would be a good dem

{'history': 'System: The human greets the AI and the AI asks what\'s up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about their schedule for the day. The AI informs them that they have a meeting at 8am with their product team, where they need to have their powerpoint presentation prepared. From 9am to 12pm, they have time to work on their LangChain project, which will go quickly due to LangChain\'s power. At noon, they have lunch at an Italian restaurant with a customer who is driving from over an hour away to learn about the latest in AI. The AI advises them to bring their laptop to show the latest LLM demo, which showcases the capabilities of their AI in translating text from one language to another with high accuracy and fluency. It\'s a great way to demonstrate the power and potential of their AI technology in the field of language translation.'}